# Grocery Sales Prediction

### Variable Description
* Item_Identifier - Unique product ID
* Item_Weight - Weight of product
* Item_Fat_Content - Whether the product is low fat or not
* Item_Visibility - The % of total display area of all products in a store allocated to the particular product
* Item_Type - The category to which the product belongs
* Item_MRP - Maximum Retail Price (list price) of the product
* Outlet_Identifier - Unique store ID
* Outlet_Establishment_Year - The year in which store was established
* Outlet_Size - The size of the store in terms of ground area covered
* Outlet_Location_Type - The type of city in which the store is located
* Outlet_Type - Whether the outlet is just a grocery store or some sort of supermarket
* Item_Outlet_Sales - Sales of the product in the particulat store. This is the outcome variable to be predicted.

### Evaluation Metric: RMSE

In [1]:
import featuretools as ft 
import numpy as np 
import pandas as pd

In [2]:
train = pd.read_csv("Grocery_Sales_Prediction/Train_Grocery.txt")
test = pd.read_csv("Grocery_Sales_Prediction/Test_Grocery.txt")

In [3]:
train.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [4]:
print(train["Item_Identifier"].unique().shape, test["Item_Identifier"].unique().shape)

(1559,) (1543,)


In [5]:
item_id = test['Item_Identifier']
store_id = test['Outlet_Identifier']
sales = train['Item_Outlet_Sales']

In [6]:
train.drop(["Item_Outlet_Sales"], axis=1, inplace=True)

In [7]:
data = pd.concat([train, test], axis=0, ignore_index=True)

In [8]:
data.isnull().sum()

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [9]:
data['Item_Weight'].fillna(data['Item_Weight'].median(), inplace = True)
data['Outlet_Size'].fillna("missing", inplace = True)

In [10]:
data["Item_Identifier"].value_counts()

FDY31    10
FDZ44    10
FDJ02    10
FDY32    10
FDN56    10
FDY08    10
FDS13    10
FDT04    10
FDM09    10
DRB25    10
FDA02    10
FDQ40    10
FDV19    10
NCF30    10
FDL58    10
FDE11    10
FDQ08    10
NCF19    10
NCQ42    10
FDY28    10
NCO54    10
NCT30    10
FDM46    10
DRH37    10
FDL38    10
FDE16    10
FDD56    10
FDX40    10
DRN47    10
NCJ19    10
         ..
FDG14     8
FDV13     8
NCR17     8
FDW37     8
NCC19     8
FDY46     8
FDE52     8
DRC13     8
FDD03     8
FDC16     8
FDU08     8
FDE20     8
FDJ52     8
FDI19     8
NCX17     8
NCS17     8
FDS43     8
NCW54     7
FDL50     7
DRN11     7
FDH58     7
FDS22     7
FDX49     7
FDM10     7
FDM52     7
FDO33     7
FDR51     7
NCL42     7
FDM50     7
FDI46     7
Name: Item_Identifier, Length: 1559, dtype: int64

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 11 columns):
Item_Identifier              14204 non-null object
Item_Weight                  14204 non-null float64
Item_Fat_Content             14204 non-null object
Item_Visibility              14204 non-null float64
Item_Type                    14204 non-null object
Item_MRP                     14204 non-null float64
Outlet_Identifier            14204 non-null object
Outlet_Establishment_Year    14204 non-null int64
Outlet_Size                  14204 non-null object
Outlet_Location_Type         14204 non-null object
Outlet_Type                  14204 non-null object
dtypes: float64(3), int64(1), object(7)
memory usage: 1.2+ MB


In [12]:
Item_fat_content_dict = {'Low Fat': 0, 'Regular': 1, 'LF': 0, 'reg': 1, 'low fat': 0}

data['Item_Fat_Content'] = data['Item_Fat_Content'].map(Item_fat_content_dict)

In [13]:
data["id"] = data["Item_Identifier"] + data["Outlet_Identifier"]
data.drop(["Item_Identifier"], axis=1, inplace=True)

# 2. Feature Tools
## 2.1. Creating Entity Set (создаем набор сущностей)
* EntitySet - это структура, которая содержит набор датафреймов и зависимостей между ними
* Наши данные содержат информацию на двух уровнях - уровень товара и уровень магазина (уровень товара - основной)
* Featuretools имеет функционал для разделения набора данных на несколько таблиц связанных таблиц
* Мы создали новую таблицу outlet из таблицы BigMart на основе id таблицы Outlet_Identifier

In [14]:
es = ft.EntitySet(id = 'sales') # creating and entity set 'es'
 
es.entity_from_dataframe(entity_id = 'bigmart', dataframe = data, index = 'id') # adding a dataframe

Entityset: sales
  Entities:
    bigmart [Rows: 14204, Columns: 11]
  Relationships:
    No relationships

In [15]:
es.normalize_entity(base_entity_id='bigmart', new_entity_id='outlet', index = 'Outlet_Identifier', 
additional_variables = ['Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type'])

Entityset: sales
  Entities:
    bigmart [Rows: 14204, Columns: 7]
    outlet [Rows: 10, Columns: 5]
  Relationships:
    bigmart.Outlet_Identifier -> outlet.Outlet_Identifier

In [16]:
print(es)

Entityset: sales
  Entities:
    bigmart [Rows: 14204, Columns: 7]
    outlet [Rows: 10, Columns: 5]
  Relationships:
    bigmart.Outlet_Identifier -> outlet.Outlet_Identifier


## 2.2 Deep Feature Synthesis

In [17]:
feature_matrix, feature_names = ft.dfs(entityset=es, 
target_entity = 'bigmart', 
max_depth = 2, 
verbose = 1, 
n_jobs = 3)

Built 37 features
EntitySet scattered to 3 workers in 1 seconds
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


In [18]:
feature_matrix.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,outlet.Outlet_Establishment_Year,outlet.Outlet_Size,outlet.Outlet_Location_Type,outlet.Outlet_Type,...,outlet.MIN(bigmart.Item_Fat_Content),outlet.MIN(bigmart.Item_Visibility),outlet.MIN(bigmart.Item_MRP),outlet.MEAN(bigmart.Item_Weight),outlet.MEAN(bigmart.Item_Fat_Content),outlet.MEAN(bigmart.Item_Visibility),outlet.MEAN(bigmart.Item_MRP),outlet.COUNT(bigmart),outlet.NUM_UNIQUE(bigmart.Item_Type),outlet.MODE(bigmart.Item_Type)
id,,,,,,,,,,,,,,,,,,,,,
DRA12OUT010,11.6,0,0.068535,Soft Drinks,143.0154,OUT010,1998,missing,Tier 3,Grocery Store,...,0,0.0,32.6558,12.722870,0.356757,0.101939,141.159742,925,16,Fruits and Vegetables
DRA12OUT013,11.6,0,0.040912,Soft Drinks,142.3154,OUT013,1987,High,Tier 3,Supermarket Type1,...,0,0.0,31.4900,12.788139,0.353509,0.060242,141.128428,1553,16,Fruits and Vegetables
DRA12OUT017,11.6,0,0.041178,Soft Drinks,140.3154,OUT017,2007,missing,Tier 2,Supermarket Type1,...,0,0.0,32.0900,12.782080,0.352560,0.061142,140.998931,1543,16,Snack Foods
DRA12OUT018,11.6,0,0.041113,Soft Drinks,142.0154,OUT018,2009,Medium,Tier 3,Supermarket Type2,...,0,0.0,31.8900,12.803638,0.353816,0.059976,141.000899,1546,16,Fruits and Vegetables
DRA12OUT027,12.6,0,0.040748,Soft Drinks,140.0154,OUT027,1985,Medium,Tier 3,Supermarket Type3,...,0,0.0,31.2900,12.600000,0.353432,0.060344,141.012347,1559,16,Fruits and Vegetables


distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/opt/conda/lib/python3.7/site-packages/distributed/client.py", line 1223, in _close
    quiet_exceptions=(CancelledError,),
  File "/opt/conda/lib/python3.7/site-packages/tornado/gen.py", line 588, in with_timeout
    chain_future(future_converted, result)
  File "/opt/conda/lib/python3.7/site-packages/tornado/concurrent.py", line 166, in chain_future
    future_add_done_callback(a, copy)
  File "/opt/conda/lib/python3.7/site-packages/tornado/concurrent.py", line 262, in future_add_done_callback
    callback(future)
  File "/opt/conda/lib/python3.7/site-packages/tornado/concurrent.py", line 160, in copy
    elif a.exception() is not None:
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback 

In [21]:
feature_matrix = feature_matrix.reindex(index=data['id'])
feature_matrix = feature_matrix.reset_index()

In [25]:
data["id"].head()

0    FDA15OUT049
1    DRC01OUT018
2    FDN15OUT049
3    FDX07OUT010
4    NCD19OUT013
Name: id, dtype: object

In [22]:
feature_matrix.head()

,id,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,outlet.Outlet_Establishment_Year,outlet.Outlet_Size,outlet.Outlet_Location_Type,...,outlet.MIN(bigmart.Item_Fat_Content),outlet.MIN(bigmart.Item_Visibility),outlet.MIN(bigmart.Item_MRP),outlet.MEAN(bigmart.Item_Weight),outlet.MEAN(bigmart.Item_Fat_Content),outlet.MEAN(bigmart.Item_Visibility),outlet.MEAN(bigmart.Item_MRP),outlet.COUNT(bigmart),outlet.NUM_UNIQUE(bigmart.Item_Type),outlet.MODE(bigmart.Item_Type)
0,FDA15OUT049,9.30,0,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,...,0,0.0,32.4558,12.803003,0.352903,0.059000,141.163199,1550,16,Fruits and Vegetables
1,DRC01OUT018,5.92,1,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,...,0,0.0,31.8900,12.803638,0.353816,0.059976,141.000899,1546,16,Fruits and Vegetables
2,FDN15OUT049,17.50,0,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,...,0,0.0,32.4558,12.803003,0.352903,0.059000,141.163199,1550,16,Fruits and Vegetables
3,FDX07OUT010,19.20,1,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,missing,Tier 3,...,0,0.0,32.6558,12.722870,0.356757,0.101939,141.159742,925,16,Fruits and Vegetables
4,NCD19OUT013,8.93,0,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,...,0,0.0,31.4900,12.788139,0.353509,0.060242,141.128428,1553,16,Fruits and Vegetables


## 2.3 Feature Primitives
Aggregation vs Transform Primitives

In [ ]:
ft.list_primitives()

In [ ]:
agg_prim_list = ["mean","max","min","std"]
trans_prim_list = ["greater_than","cum_count"]

## 2.4. Create Custom Primitives

In [ ]:
from featuretools.primitives import make_agg_primitive, make_trans_primitive

from featuretools.variable_types import Text, Numeric

def absolute(column):
    
    return abs(column)

Absolute = make_trans_primitive(function=absolute, 
                                input_types=[Numeric],
                                return_type=Numeric)

def maximum(column):
    return max(column)

Maximum = make_agg_primitive(function=maximum,
                             input_types=[Numeric],
                             return_type=Numeric)

In [ ]:
feature_matrix, feature_names = ft.dfs(entityset=es, 
                                       
target_entity = 'bigmart', 
                                       
agg_primitives=agg_prim_list,
                                       
trans_primitives=trans_prim_list,      
                                       
max_depth = 2, # Maximum allowed depth of features
verbose = 1, 
n_jobs = 3)

## 3. CatBoost Model

In [26]:
from catboost import CatBoostRegressor

In [27]:
categorical_features = np.where(feature_matrix.dtypes == 'object')[0]

for i in categorical_features:
    feature_matrix.iloc[:,i] = feature_matrix.iloc[:,i].astype('str')

In [28]:
feature_matrix.drop(['id'], axis=1, inplace=True)
train = feature_matrix[:8523]
test = feature_matrix[8523:]

In [29]:
# removing uneccesary variables
train.drop(['Outlet_Identifier'], axis=1, inplace=True)
test.drop(['Outlet_Identifier'], axis=1, inplace=True)

In [30]:
# identifying categorical features
categorical_features = np.where(train.dtypes == 'object')[0]

In [31]:
categorical_features

array([ 3,  6,  7,  8, 35])

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train, sales, test_size=0.25, random_state=42)

In [33]:
model_cat = CatBoostRegressor(iterations=100, learning_rate=0.3, depth=6, eval_metric='RMSE', random_seed=42)

In [34]:
model_cat.fit(X_train, y_train, cat_features=categorical_features, use_best_model=True)

You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 2149.5890687	total: 96ms	remaining: 9.5s
1:	learn: 1729.8585871	total: 143ms	remaining: 7.02s
2:	learn: 1461.5789786	total: 190ms	remaining: 6.14s
3:	learn: 1294.8689567	total: 235ms	remaining: 5.63s
4:	learn: 1198.8568160	total: 287ms	remaining: 5.45s
5:	learn: 1146.5921927	total: 345ms	remaining: 5.4s
6:	learn: 1118.8689859	total: 391ms	remaining: 5.19s
7:	learn: 1102.4288564	total: 441ms	remaining: 5.07s
8:	learn: 1092.4493587	total: 488ms	remaining: 4.94s
9:	learn: 1085.5522509	total: 541ms	remaining: 4.87s
10:	learn: 1081.6364207	total: 560ms	remaining: 4.53s
11:	learn: 1079.3234010	total: 572ms	remaining: 4.19s
12:	learn: 1077.0471587	total: 586ms	remaining: 3.92s
13:	learn: 1076.3564973	total: 613ms	remaining: 3.76s
14:	learn: 1075.6832880	total: 628ms	remaining: 3.56s
15:	learn: 1074.7067306	total: 644ms	remaining: 3.38s
16:	learn: 1073.9721471	total: 665ms	remaining: 3.25s
17:	learn: 1072.7053397	total: 680ms	remaining: 3.1s
18:	learn: 1071.5775344	total: 690ms	remai

In [42]:
y_pred = model_cat.predict(X_valid)

In [51]:
rmse = np.sqrt(np.mean((y_pred - y_valid)**2))

In [52]:
print(rmse)

1069.199923162786


In [62]:
y_test = pd.DataFrame(model_cat.predict(test), columns=["submit"])

In [63]:
y_test.to_csv("submit.csv")

In [64]:
y_test

,submit
0,1522.307722
1,1408.340544
2,613.173722
3,2313.985723
4,6534.341154
5,1841.839254
6,660.348037
7,2333.031769
8,1507.630690
9,3085.014537
